In [1]:
import numpy as np

# Time Wrapper

In [2]:
import time
from functools import wraps


def timing_decorator(func):
    """Decorator that measures and prints average execution time over 100000 iterations."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        total_execution_time = 0
        iterations = 100000

        for _ in range(iterations):
            start_time = time.perf_counter()
            result = func(*args, **kwargs)
            end_time = time.perf_counter()
            total_execution_time += (end_time - start_time)

        average_execution_time = total_execution_time / iterations
        print(
            f"Average execution time over {iterations} iterations: {average_execution_time:.8f} seconds")

        return result

    return wrapper

## Mean|

In [3]:
@timing_decorator
def custom_mean(x):
    """Compute mean as sum divided by size."""
    return np.sum(x)/x.size


@timing_decorator
def mean(x):
    """NumPy mean wrapper."""
    return np.mean(x)

In [4]:
arr = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [5]:
custom_mean(arr)

Average execution time over 100000 iterations: 0.00000275 seconds


np.float64(5.5)

In [6]:
mean(arr)

Average execution time over 100000 iterations: 0.00000414 seconds


np.float64(5.5)

# Var

In [7]:
def custom_mean(x):
    """Compute mean as sum divided by size."""
    return np.sum(x)/x.size

In [8]:
@timing_decorator
def custom_var(x):
    """Compute variance using E[X^2] - E[X]^2."""
    mean_x = custom_mean(x)
    return custom_mean(x**2) - mean_x**2


@timing_decorator
def custom_var_2(x):
    """Compute variance as mean of squares minus square of mean."""
    return custom_mean(x**2) - custom_mean(x)**2


@timing_decorator
def custom_var_3(x):
    """Compute variance using squared deviation from mean."""
    return custom_mean(np.square(x-custom_mean(x)))


@timing_decorator
def var(x):
    """NumPy variance wrapper."""
    return np.var(x)

In [9]:
@timing_decorator
def custom_var_optimized(x):
    """Compute biased variance in a single pass."""
    x_squared_sum = np.sum(x**2)
    x_sum = np.sum(x)
    x_size = x.size
    return (x_squared_sum / x_size) - (x_sum / x_size)**2

In [10]:
@timing_decorator
def custom_unbias_var_optimized(x):
    """Compute unbiased variance (N-1 denominator) in a single pass."""
    x_squared_sum = np.sum(x**2)
    x_sum = np.sum(x)
    x_size = x.size-1
    return (x_squared_sum / x_size) - (x_sum / x_size)**2

In [11]:
custom_unbias_var_optimized(arr)

Average execution time over 100000 iterations: 0.00000608 seconds


np.float64(5.432098765432102)

In [12]:
custom_var(arr)

Average execution time over 100000 iterations: 0.00000608 seconds


np.float64(8.25)

In [13]:
custom_var_optimized(arr)

Average execution time over 100000 iterations: 0.00000591 seconds


np.float64(8.25)

In [14]:
custom_var_2(arr)

Average execution time over 100000 iterations: 0.00000613 seconds


np.float64(8.25)

In [15]:
custom_var_3(arr)

Average execution time over 100000 iterations: 0.00000705 seconds


np.float64(8.25)

In [16]:
var(arr)

Average execution time over 100000 iterations: 0.00001082 seconds


np.float64(8.25)

# Std

In [17]:
def custom_var_optimized(x):
    """Compute unbiased variance (N-1 denominator) in a single pass."""
    x_squared_sum = np.sum(x**2)
    x_sum = np.sum(x)
    x_size = x.size-1
    return (x_squared_sum / x_size) - (x_sum / x_size)**2

In [18]:
@timing_decorator
def custom_std(x):
    """Compute standard deviation as sqrt of custom variance."""
    return np.sqrt(custom_var_optimized(x))


@timing_decorator
def std(x):
    """NumPy standard deviation wrapper."""
    return np.std(x)

In [19]:
std(arr)

Average execution time over 100000 iterations: 0.00001209 seconds


np.float64(2.8722813232690143)

In [20]:
custom_std(arr)

Average execution time over 100000 iterations: 0.00000678 seconds


np.float64(2.330686329267004)

# Median

In [21]:
@timing_decorator
def custom_median(x):
    """Compute median via sort and middle index(es)."""
    x_sort = np.sort(x)
    n = x.size
    if n%2 == 1:
        return x_sort[n//2]
    else:
        m1 = x_sort[n//2]
        m2 = x_sort[n//2-1]
        return (m1+m2)/2

In [22]:
@timing_decorator
def median(x):
    """NumPy median wrapper."""
    return np.median(x)

In [23]:
median(arr)

Average execution time over 100000 iterations: 0.00000910 seconds


np.float64(5.5)

In [24]:
custom_median(arr)

Average execution time over 100000 iterations: 0.00000161 seconds


np.float64(5.5)

# Mode

In [25]:
@timing_decorator
def custom_mode(x):
    """Compute mode via bincount and argmax (1D arrays only)."""
    if len(x.shape) != 1:
        raise ValueError(f"custom_mode requires 1D array, got shape {x.shape}")

    counts = np.bincount(x)

    mode_value = np.argmax(counts)

    return mode_value

In [26]:
from scipy import stats


@timing_decorator
def mode(x):
    """SciPy mode wrapper."""
    return stats.mode(x)

In [27]:
# mode(arr)

In [28]:
custom_mode(arr)

Average execution time over 100000 iterations: 0.00000220 seconds


np.int64(1)

# Percentile

In [29]:
# @timing_decorator
def custom_percentile(x, percentile):
    """Compute percentile via sort and linear interpolation."""
    # Step 1: Sorting the data
    x_sort = np.sort(x)

    # Step 2: Finding the index of the percentile value
    index = (percentile / 100) * (x_sort.size - 1)

    # Step 3: Interpolating to get the percentile value
    lower = np.floor(index).astype(int)  # Finding the lower index
    upper = np.ceil(index).astype(int)   # Finding the upper index
    if lower == upper:  # If index is an integer
        return x_sort[lower]  # Return the value at index
    else:
        # Interpolate between the values at the lower and upper indices
        return x_sort[lower] + (x_sort[upper] - x_sort[lower]) * (index - lower)


In [30]:
@timing_decorator
def percentile(x, percentile):
    """NumPy percentile wrapper."""
    return np.percentile(x, percentile)

In [31]:
custom_percentile(arr, percentile=20)

np.float64(2.8)

In [32]:
percentile(arr, percentile=20)

Average execution time over 100000 iterations: 0.00004218 seconds


np.float64(2.8)